reference: http://trap.ncirl.ie/1845/1/briangibbons.pdf

https://www.reddit.com/r/FantasyPL/comments/c64rrx/fpl_api_url_has_been_changed/

# 1. Import Packages

In [1]:
# Working with urls
from urllib.request import urlopen   # https://docs.python.org/3/library/urllib.html

# JSON encoder and decoder
import json # https://docs.python.org/3/library/json.html

import pandas as pd

# to read/write Excel 2010 xlsx/xlsm/xltx/xltm files.
from openpyxl import load_workbook

# An enhanced, interactive shell for Python.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# sets the backend of matplotlib to the 'inline' backend
%matplotlib inline

# 2. Help Fuction

In [2]:
# This function helps to write a dataframe to a csv.
def writing_in_excel(path, df, sheet_name):
    writer = pd.ExcelWriter(path, datetime_format='m/d/yyyy')
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

# 3. Test if crawler works

### Info about bootstrap

** Bootstrop is just the overall data for each player -- not each game.**

In [8]:
# info about bootstrap (overall)
url_to_bootstrap = "https://fantasy.premierleague.com/api/bootstrap-static/"
bootstrap_html = urlopen(url_to_bootstrap) # connect to the url
bootstrap = json.load(bootstrap_html) # read the json data in the url

### Info about a single player

** This will give us the data of how a player performed each game.**

In [9]:
# this demonstration will result how well the player 1 performed each game.

player_id = str(1)
player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/" + player_id + '/')
player_raw = json.load(player_html)

In [10]:
# both bootstrap and player data is in dictionary, so keys() will show what kind of data we can get from each dictionary.

bootstrap.keys()
player_raw.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

dict_keys(['fixtures', 'history', 'history_past'])

### Info about my team

** This will give me which players I have in my team now **

In [16]:
import requests
session = requests.session()

url = 'https://users.premierleague.com/accounts/login/'
payload = {
 'password': 'Praisehim1223',
 'login': 'hyt037@gmail.com',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(url, data=payload)

<Response [200]>

In [29]:
response = session.get('https://fantasy.premierleague.com/api/my-team/1149818')
myteam = response.json()
myteam.keys()

dict_keys(['picks', 'chips', 'transfers'])

In [37]:
roaster = []
for i in range(len(myteam['picks'])):
    roaster.append(myteam['picks'][i]['element'])
    
roaster

[189, 182, 105, 183, 118, 368, 265, 29, 211, 91, 210, 93, 159, 141, 396]

### First, check each keys from the bootstrap data to see how they look like

In [7]:
events = pd.DataFrame(bootstrap['events']) #


#game_settings = pd.DataFrame(bootstrap['game_settings'])
phases =  pd.DataFrame(bootstrap['phases']) 
teams =  pd.DataFrame(bootstrap['teams']) 
#total_players = pd.DataFrame(bootstrap['total_players'])
elements =  pd.DataFrame(bootstrap['elements']) 
element_stats =  pd.DataFrame(bootstrap['element_stats'])
element_types =  pd.DataFrame(bootstrap['element_types']) # positions

fixtures = pd.DataFrame(player_raw['fixtures']) #
history = pd.DataFrame(player_raw['history'])
history_past = pd.DataFrame(player_raw['history_past'])

### Then, check from the player data

In [8]:
##### From Player_raw ######

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past =  pd.DataFrame(player_raw['history_past']) 
    
fixtures['element'] = 1

fixtures.iloc[:1] # we only want to see the very next fixture. 

,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,element
0,1059943,5,None,1,None,25.0,False,0,False,2020-02-02T14:00:00Z,Gameweek 25,False,3,1


# 4. Defining a scraper

***put everything from the section 3 above to define functions***

In [9]:
def bootstrap_scraper():
    '''this function will scrape the bootstrap data and save them as csv.'''
    
    # info about bootstrap (overall)
    bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/") # connect to the bootstrap url
    bootstrap = json.load(bootstrap_html) # read the json.
    
    '''get the data we want from the boostrap'''
    events = pd.DataFrame(bootstrap['events'])
    #game_settings = pd.DataFrame(bootstrap['game_settings'])
    phases =  pd.DataFrame(bootstrap['phases']) 
    teams =  pd.DataFrame(bootstrap['teams']) 
    #total_players = pd.DataFrame(bootstrap['total_players'])
    elements =  pd.DataFrame(bootstrap['elements']) 
    element_stats =  pd.DataFrame(bootstrap['element_stats']) 
    element_types =  pd.DataFrame(bootstrap['element_types'])
    
    
    '''save the data we scraped.'''
    select = 'D' # Adjust this select variable. I did this because I have D drive for my laptop and F for my desktop. so I can just switch between D and F easily in this way depends on which device I am working on.

    #events.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019//events_df.csv')
    #game_settings.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\game_settings_df.xlsx')
    #phases.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\phases_df.csv')
    teams.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\teams_df.csv')
    elements.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\elements_df.csv')
    #element_stats.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_stats_df.csv')
    element_types.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_types_df.csv')


In [10]:
def player_data_scraper():
    '''This function will iterate through all the players in the database and scrape the data. Then, merge them.'''
    
    player_id = 1 # starting point
    
    history_list = [] # history data will be stored in this list
    fixtures_list = [] # fixture data will be stored in this list
    
    while player_id < 600: # I found out there are less than 600 players in the database.
        try:
            player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/" + str(player_id) + '/') # connect to the url of a player         
            player_raw = json.load(player_html) # read the json file.
                   
            history = pd.DataFrame(player_raw['history']) # create a dataframe of the history data
            fixtures = pd.DataFrame(player_raw['fixtures']) # create a dataframe of the fixture data

            
            fixtures['element'] = player_id # we only need the fixture of this particular player
            
            
            history_list.append(history) # append the history data to the list.
            fixtures_list.append(fixtures.iloc[:1]) # append the very next fixture to the fixture list.
            
            
            history_df = pd.concat(history_list) # merge all the data in the history list
            history_df = pd.DataFrame(history_df) # then, create the dataframe 
            
            fixtures_df = pd.concat(fixtures_list) # merge all the data in the fixture list
            fixtures_df = pd.DataFrame(fixtures_df) # then, create the dataframe 
            
        except:
            pass
            
    
        print (player_id) # print which player it just went through
        player_id += 1 # move on to the next player
    
    return history_df, fixtures_df
    

# Run the scrapers

In [11]:
bootstrap_scraper() # this automatically saves the data

In [12]:
history_df, fixtures_df = player_data_scraper()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
select = 'D'
history_df.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\history_df.csv')
fixtures_df.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\fixture_df.csv')


# Debugging
